# [作者 TsukiSama9292 - Github 連結](https://github.com/tsukisama9292)
## [參考文獻 - 連結](https://ianwu.tw/press/topic/command_line_program/yt-dlp.html#%E4%BB%8B%E7%B4%B9)

# 依賴安裝 - 沒有 ffmpeg 的話，只能做音檔下載

In [ ]:
!pip -qqq install yt-dlp

# 音檔下載，YOUTUBE_URL 換成 清單或影片 的連結 都可以

In [ ]:
import zipfile
import os

# 請改變YOUTUBE_URL為想要的Youtube影片連結或清單連結(不能是分享連結)
YOUTUBE_URL = ''

# 執行下載，底下的程式碼可以不改
!yt-dlp -o 'Download/%(title)s.%(ext)s' --extract-audio --audio-format mp3 {YOUTUBE_URL}
folder_to_zip = 'Download'
zip_filename = 'Download_MP3.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, dirs, files in os.walk(folder_to_zip):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_to_zip))

print(f'資料夾 {folder_to_zip} 已成功壓縮為 {zip_filename}')
print(f'已經可以下載{zip_filename}到本機端')
!rm -rf Download

# 影片下載 - 推薦的使用方法

In [ ]:
!sudo apt update
!sudo apt install ffmpeg

In [ ]:
# 請改變YOUTUBE_URL為想要的Youtube影片連結或清單連結(不能是分享連結)
YOUTUBE_URL = ''
### 確認支援的 mp4 解析度 與 audio only 音質 的 ID
!yt-dlp -F {YOUTUBE_URL}

In [ ]:
# 執行下載，遇到問題再改ID組合，當前616為mp4，233為m4a，底下的程式碼可以不改
!yt-dlp -f '616+233' -o 'Download/%(title)s.%(ext)s' --recode mp4 {YOUTUBE_URL}
folder_to_zip = 'Download'
zip_filename = 'Download_MP4.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, dirs, files in os.walk(folder_to_zip):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_to_zip))

print(f'資料夾 {folder_to_zip} 已成功壓縮為 {zip_filename}')
print(f'已經可以下載{zip_filename}到本機端')
!rm -rf Download